## Task about sum and product 

Note: This is optional task. 
However it is also evaluated and so you will get extra points to final score. 
Thus, we encourage you to solve it. 

```
There are two whole numbers:
1 < a,b <100

One scientist("Sum") get provided with sum of numbers,
another  ("Prod") get provided with product of numbers. 
Both scientists know that numbers 1 < a,b <100.

Determine the numbers being based on the following dialog: 
    Prod: I don't know the numbers;
    Sum: I know it;
    Prod: then I know the numbers; 
    Sum: then I know the numbers too.
```

In [58]:
# unique_prods = both numbers prime but not only this: actually, we just need to check this for each product

# Prod: I don't know the numbers; >>> the product is not unique

# Sum: I know it; >>> cannot be a sum of numbers, that make a unique product

# Prod: then I know the numbers; >>> if divided on factors there is only one option
#                                    of not being a sum of prime numbers

# Sum: then I know the numbers too. >>> if divided on summands there is only one option 
#                                       that leaves Prod with only one option on previous step

# The above looks quite complicated and layered, and it seems that there will be a lot of different lists
# to be managed in process, e.g. lists for all possible products, sums, sets of factors, prime numbers.
# But thinking of a way to make the solution more readable and efficient, using numpy matrices comes to mind.
# If we create one matrix for products and one for sums, where the indices correspond to the numbers in range(2,100),
# then we can conveniently exclude the options from both matrices by indices or values, step by step.
# I assume that when we complete all the steps, there will be only one pair of indices left in both matrices.
# And that combination of numbers will be the answer.

# Some additional notes:
# - The products and sums matrices are symmetric, 
#   so it's enough to use the upper triangles only.
# - Deleting an option by value from sums matrix means
#   deleting the whole opposite diagonal with that value

In [59]:
import numpy as np
# import itertools

# def list_primes(upto):
#     odds = np.arange(3, upto + 1, 2)
#     isprime = np.ones((upto - 1) // 2, dtype=bool)
#     for factor in odds[:int(upto**.5)//2]:
#         if isprime[(factor-2)//2]:
#             isprime[(factor*3-2)//2::factor] = 0
#     return np.insert(odds[isprime], 0, 2)

# primes = np.array(list_primes(99))
numbers = np.arange(2, 100)

products = np.zeros((100, 100), dtype=int)
products[2:, 2:] = np.triu(numbers[:, None] * numbers[None, :])

products_original = products.copy()

sums = np.zeros((100, 100), dtype=int)
sums[2:, 2:] = np.triu(numbers[:, None] + numbers[None, :])

print(products, '\n')
print(sums)

[[   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    4 ...  194  196  198]
 ...
 [   0    0    0 ... 9409 9506 9603]
 [   0    0    0 ...    0 9604 9702]
 [   0    0    0 ...    0    0 9801]] 

[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   4 ...  99 100 101]
 ...
 [  0   0   0 ... 194 195 196]
 [  0   0   0 ...   0 196 197]
 [  0   0   0 ...   0   0 198]]


In [60]:
def remove_diagonal(x, y, *, mark=0):
    '''Removes the whole opposite diagonal in upper triangle from products and sums matrices'''
    rng = np.arange(max(x + y - 99, 0), min(x + y + 1, 100))
    upper_triangle_diagonal = rng[:(len(rng)+1)//2], rng[:-(len(rng)+2)//2:-1]

    sums[upper_triangle_diagonal] = mark
    products[upper_triangle_diagonal] = mark
    
def number_available():
    '''Returns the number of available values for the next step'''
    return np.sum(sums > 0)

'Total available pairs of numbers: ', number_available()

('Total available pairs of numbers: ', 4851)

In [61]:
# Prod: I don't know the numbers; >>> product is not unique
# Step 1. Exclude all values that do not have duplicates in upper triangle of the products matrix

# Sum: I know it; >>> cannot be a sum of numbers that make a unique product
# Step 2. Exclude all values that can be formed by sum of numbers that make a unique product
# i.e exclude opposite diagonal for each where value from Step 1.

# commented below is insufficient
# prime_combos = itertools.combinations_with_replacement(primes, 2)
# for combo in prime_combos:
#     remove_diagonal(*combo, mark=-1) # marking prime sums diagonals for step 3

unique_prods, unique_counts = np.unique(products, return_counts=True)
unique_prods = unique_prods[unique_counts == 1]
unique_indices = np.where(np.isin(products, unique_prods))
for x, y in zip(*unique_indices):
    remove_diagonal(x, y, mark=-1) # marking prime sums diagonals for step 3

'Available pairs of numbers after Step 1 and Step 2: ', number_available()

('Available pairs of numbers after Step 1 and Step 2: ', 145)

In [62]:
# Prod: then I know the numbers; >>> if divided on factors there is only one option 
#                                    of not being a sum of prime numbers
# Step 3. For each product find indices of equal values, count pairs which make sums
#         that lay in a diagonal of a prime number sum. Than filter out single counts
#         but only when there are multiple sets of factors (else prod could have guessed earlier).

unique_prods = np.unique(products)[2:]
print('Number of unique products processed on Step 3: ', unique_prods.shape)
good_prods = np.array([])
for prod in unique_prods:
    corresponding_sums = sums[np.where(products_original == prod)]
    if len(corresponding_sums) > 1 and np.sum(corresponding_sums != -1) == 1:
        good_prods = np.append(good_prods, prod)
        products[np.where(products_original == prod)] = -2 # marking appropriate products for step 4

print('Number of available products after Step 3: ', len(good_prods))

Number of unique products processed on Step 3:  (113,)
Number of available products after Step 3:  86


In [63]:
# Sum: then I know the numbers too. >>> if divided on summands there is only one option 
#                                       that leaves Prod with only one option on previous step
# Step 4. Divide each sum on summands, count combos that multiply into one of prods values
#         from Step 3. Then filter out single counts. There sould be only one such. 

unique_sums = np.unique(sums)[2:]
print('Number of unique sums processed on Step 4: ', unique_sums.shape)
good_sums = np.array([])
solutions = np.empty((0, 2), int)
for sum in unique_sums:
    corresponding_prods = products[np.where(sums == sum)]
    if np.sum(corresponding_prods == -2) == 1:
        good_sums = np.append(good_sums, sum)
        # these will always be single value arrays
        solution = np.array(np.where(np.logical_and(sums == sum, products == -2))).reshape(2)
        solutions = np.append(solutions, [solution], axis=0)
print('Number of available sums after Step 4: ', len(good_sums))
print('Corresponging pairs of numbers: \n', solutions)

Number of unique sums processed on Step 4:  (10,)
Number of available sums after Step 4:  1
Corresponging pairs of numbers: 
 [[ 4 13]]


### Final note.
Since we are getting multiple possible pairs of numbers, then this task cannot be logically completed and there is no right solution for these particular conditions. BUT! If we cut down our set of available sums so they cannot reach 69 + 96 = 162 (a + b < 162), then all the solutions except first one will not be available. So in that case the task will be solvable and such dialog will make sence. In fact, it turns out the classical version of this problem has a restriction that says a + b < 100. It's dated 1969 and is called [Impossible Puzzle](https://en.wikipedia.org/wiki/Sum_and_Product_Puzzle) or Freudenthal Problem. Nice!

In [64]:
if len(good_sums) == 1:
    print('Solution found! Numbers are: ', solutions[0])
elif len(good_sums) > 1:
    print('Multiple solutions found. Thus, scientists are not honest! Solutions: \n', *solutions)
else:
    print('No solutions found. Thus, scientists are not honest!')

Solution found! Numbers are:  [ 4 13]
